In [ ]:
import glob
import os
import pickle
import pprint
from datetime import datetime
from pathlib import Path

import cv2
import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pytz
import seaborn as sns
from omegaconf import OmegaConf
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

from src.config import cfg
from src.dir import create_dir
from src.seed import seed_everything

cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)
pl.Config.set_fmt_str_lengths(100)


In [2]:
DEBUG = False

if DEBUG:
    cfg.lgb.num_boost_round = 10


### データの読み込み

In [3]:
# データの読み込み
train = pl.read_csv(cfg.data.train_path, try_parse_dates=True)
test = pl.read_csv(cfg.data.test_path, try_parse_dates=True)
sample_submission = pl.read_csv(cfg.data.sample_submission_path, try_parse_dates=True)

# 前処理

# indexを作成（学習、推論前に元の順序情報を復元するため）
train = train.with_row_index()
test = test.with_row_index()

# bool型をint8に変換
train = train.with_columns(train.select(pl.col(pl.Boolean).cast(pl.Int8)))
test = test.with_columns(test.select(pl.col(pl.Boolean).cast(pl.Int8)))

# scene列を作成 → これでGroupKFoldする
train = train.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))
test = test.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))

# decisecond列を作成
train = train.with_columns(pl.col("ID").str.split("_").list[1].cast(pl.Int32).alias("decisecond"))
test = test.with_columns(pl.col("ID").str.split("_").list[1].cast(pl.Int32).alias("decisecond"))

# trainとtestを結合する（ラベルエンコーディング用）
train_test = pl.concat([train, test], how="diagonal")

# CV
gkf = GroupKFold(n_splits=cfg.n_splits)


In [ ]:
train.head()


In [5]:
def extract_image_features(img_dir, depth_dir):
    features = {}
    sec_list = ["t-1.0", "t-0.5", "t"]

    # 3枚の画像を読み込みとDepth画像を読み込み
    images = []
    depths = []
    for sec in sec_list:
        img_path = Path(img_dir) / f"image_{sec}.png"
        depth_path = Path(depth_dir) / f"image_{sec}.png"
        img = cv2.imread(str(img_path))
        depth = cv2.imread(str(depth_path))
        images.append(img)
        depths.append(depth)

    # RGB画像とDepth画像の両方に対して特徴抽出
    for sec, img, depth in zip(sec_list, images, depths, strict=True):
        # RGB画像の基本統計量
        features[f"variance_{sec}"] = np.var(img)
        features[f"mean_brightness_{sec}"] = np.mean(img)

        # Depth画像の基本統計量
        features[f"variance_depth_{sec}"] = np.var(depth)
        features[f"mean_depth_{sec}"] = np.mean(depth)

        # RGB画像のエッジ特徴
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        features[f"edge_density_{sec}"] = np.mean(edges > 0)

        # Depth画像のエッジ特徴
        depth_gray = cv2.cvtColor(depth, cv2.COLOR_BGR2GRAY)
        depth_edges = cv2.Canny(depth_gray, 100, 200)
        features[f"depth_edge_density_{sec}"] = np.mean(depth_edges > 0)

        # RGB画像の色相ヒストグラム
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0], None, [6], [0, 180])
        for bin_idx in range(6):
            features[f"hue_bin_{sec}_{bin_idx}"] = hist[bin_idx][0]

        # RGB画像の上部/下部での特徴の違い
        height = img.shape[0]
        upper_half = img[: height // 2, :, :]
        lower_half = img[height // 2 :, :, :]
        features[f"upper_brightness_{sec}"] = np.mean(upper_half)
        features[f"lower_brightness_{sec}"] = np.mean(lower_half)
        features[f"brightness_ratio_{sec}"] = features[f"upper_brightness_{sec}"] / (
            features[f"lower_brightness_{sec}"] + 1e-6
        )

        # Depth画像の上部/下部での特徴の違い
        depth_upper = depth[: height // 2, :, :]
        depth_lower = depth[height // 2 :, :, :]
        features[f"depth_upper_mean_{sec}"] = np.mean(depth_upper)
        features[f"depth_lower_mean_{sec}"] = np.mean(depth_lower)
        features[f"depth_vertical_ratio_{sec}"] = features[f"depth_upper_mean_{sec}"] / (
            features[f"depth_lower_mean_{sec}"] + 1e-6
        )

        # RGB画像の左右での特徴の違い
        width = img.shape[1]
        left_half = img[:, : width // 2, :]
        right_half = img[:, width // 2 :, :]
        features[f"left_brightness_{sec}"] = np.mean(left_half)
        features[f"right_brightness_{sec}"] = np.mean(right_half)
        features[f"lr_brightness_ratio_{sec}"] = features[f"left_brightness_{sec}"] / (
            features[f"right_brightness_{sec}"] + 1e-6
        )

        # Depth画像の左右での特徴の違い
        depth_left = depth[:, : width // 2, :]
        depth_right = depth[:, width // 2 :, :]
        features[f"depth_left_mean_{sec}"] = np.mean(depth_left)
        features[f"depth_right_mean_{sec}"] = np.mean(depth_right)
        features[f"depth_horizontal_ratio_{sec}"] = features[f"depth_left_mean_{sec}"] / (
            features[f"depth_right_mean_{sec}"] + 1e-6
        )

        # RGB画像のテクスチャ特徴
        features[f"texture_contrast_{sec}"] = np.std(gray)

        # Depth画像のテクスチャ特徴
        features[f"depth_texture_contrast_{sec}"] = np.std(depth_gray)

        # RGB画像のエッジの方向性
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        features[f"edge_direction_{sec}"] = np.mean(np.arctan2(sobely, sobelx))

        # Depth画像のエッジの方向性と勾配強度
        depth_sobelx = cv2.Sobel(depth_gray, cv2.CV_64F, 1, 0, ksize=3)
        depth_sobely = cv2.Sobel(depth_gray, cv2.CV_64F, 0, 1, ksize=3)
        features[f"depth_gradient_direction_{sec}"] = np.mean(np.arctan2(depth_sobely, depth_sobelx))
        features[f"depth_gradient_magnitude_{sec}"] = np.mean(np.sqrt(depth_sobelx**2 + depth_sobely**2))

        # Depth画像の四分位数特徴
        depth_quartiles = np.percentile(depth_gray, [25, 50, 75])
        features[f"depth_q1_{sec}"] = depth_quartiles[0]
        features[f"depth_q2_{sec}"] = depth_quartiles[1]
        features[f"depth_q3_{sec}"] = depth_quartiles[2]
        features[f"depth_iqr_{sec}"] = depth_quartiles[2] - depth_quartiles[0]

    # 画像間の差分特徴（RGB画像とDepth画像の両方）
    for i in range(2):
        # RGB画像の差分
        rgb_diff = cv2.absdiff(images[i], images[i + 1])
        features[f"frame_diff_{sec_list[i]}_{sec_list[i + 1]}"] = np.mean(rgb_diff)

        # Depth画像の差分
        depth_diff = cv2.absdiff(depths[i], depths[i + 1])
        features[f"depth_diff_{sec_list[i]}_{sec_list[i + 1]}"] = np.mean(depth_diff)

        # Depth画像の差分の統計量
        features[f"depth_diff_std_{sec_list[i]}_{sec_list[i + 1]}"] = np.std(depth_diff)
        features[f"depth_diff_max_{sec_list[i]}_{sec_list[i + 1]}"] = np.max(depth_diff)

    return features


In [ ]:
# 画像特徴量のdfへの結合
def add_image_features(df):
    all_features = []
    for id_name in tqdm(df["ID"]):
        img_dir = Path(cfg.data.img_root) / id_name
        depth_dir = Path(cfg.data.depth_root) / id_name

        features = extract_image_features(img_dir, depth_dir)
        features["ID"] = id_name
        all_features.append(features)

    # 画像特徴量のdfを作成
    img_features_df = pl.DataFrame(all_features)

    # 元のdfと結合
    return df.join(img_features_df, how="left", on="ID")


# trainに画像特徴量を追加
train = add_image_features(train)

# testに画像特徴量を追加
test = add_image_features(test)


In [7]:
# exp011のoofとpredictionsを特徴量として追加(train, testのデータの順序が変わっていないか注意)
oof_exp011 = np.load("../../results/011/20241125_171020/oof_predictions.npy")
predictions_exp011 = np.load("../../results/011/20241125_171020/final_predictions.npy")

# trainにoofを追加
exprs = []
exprs += [pl.Series(oof_exp011[:, i]).alias(f"exp011_nn_{cfg.target_cols[i]}") for i in range(len(cfg.target_cols))]
train = train.with_columns(exprs)

# testにpredictionsを追加
exprs = []
exprs += [
    pl.Series(predictions_exp011[:, i]).alias(f"exp011_nn_{cfg.target_cols[i]}") for i in range(len(cfg.target_cols))
]
test = test.with_columns(exprs)


In [8]:
# 物理量の組合せ特徴量

# 速度と加速度の組み合わせ
train = train.with_columns(
    [
        (pl.col("vEgo") * pl.col("aEgo")).alias("velocity_acceleration"),
        (pl.col("vEgo") * pl.col("steeringAngleDeg")).alias("velocity_steering"),
        (pl.col("aEgo") * pl.col("steeringAngleDeg")).alias("acceleration_steering"),
    ]
)
test = test.with_columns(
    [
        (pl.col("vEgo") * pl.col("aEgo")).alias("velocity_acceleration"),
        (pl.col("vEgo") * pl.col("steeringAngleDeg")).alias("velocity_steering"),
        (pl.col("aEgo") * pl.col("steeringAngleDeg")).alias("acceleration_steering"),
    ]
)

# 二乗項（非線形性の捕捉）
train = train.with_columns(
    [
        (pl.col("vEgo") ** 2).alias("vEgo_squared"),
        (pl.col("aEgo") ** 2).alias("aEgo_squared"),
        (pl.col("steeringAngleDeg") ** 2).alias("steering_squared"),
    ]
)
test = test.with_columns(
    [
        (pl.col("vEgo") ** 2).alias("vEgo_squared"),
        (pl.col("aEgo") ** 2).alias("aEgo_squared"),
        (pl.col("steeringAngleDeg") ** 2).alias("steering_squared"),
    ]
)


In [9]:
# gearShifterのラベルエンコーディング
le = LabelEncoder()
le.fit(train_test["gearShifter"].fill_null(""))
train = train.with_columns(
    pl.Series(le.transform(train["gearShifter"].fill_null(""))).cast(pl.Int16).alias("gearShifter")
)
test = test.with_columns(pl.Series(le.transform(test["gearShifter"].fill_null(""))).cast(pl.Int16).alias("gearShifter"))
train_test = pl.concat([train, test], how="diagonal")


In [ ]:
# シーンごとのデータ数の分布 → 多くのシーンが3以上あるので2個前後の値の特徴量があってもいいと思う
data = train_test.group_by("scene").agg(pl.len()).get_column("len")
plt.figure(figsize=(8, 4))
plt.hist(data, bins=30)
plt.xlabel("シーンごとのデータ数")
plt.ylabel("シーン数")
plt.title("シーンごとのデータ数の分布")
plt.grid(True)
plt.show()


In [11]:
# 同一シーンからの集約特徴量作成

agg_cols = ["vEgo", "aEgo", "steeringAngleDeg", "steeringTorque", "gas"]
bool_cols = ["brakePressed", "gasPressed", "leftBlinker", "rightBlinker"]
cat_cols = ["gearShifter"]

exprs = []
exprs += [
    pl.col(agg_col).shift(-2).over("scene").alias(f"{agg_col}_shift-2") for agg_col in agg_cols + bool_cols + cat_cols
]  # 2ステップ前の時間の値
exprs += [
    pl.col(agg_col).shift(2).over("scene").alias(f"{agg_col}_shift2") for agg_col in agg_cols + bool_cols + cat_cols
]  # 2ステップ後の時間の値
exprs += [
    pl.col(agg_col).shift(-1).over("scene").alias(f"{agg_col}_shift-1") for agg_col in agg_cols + bool_cols + cat_cols
]  # 1ステップ前の時間の値
exprs += [
    pl.col(agg_col).shift(1).over("scene").alias(f"{agg_col}_shift1") for agg_col in agg_cols + bool_cols + cat_cols
]  # 1ステップ後の時間の値
exprs += [
    pl.col(agg_col).diff(-1).over("scene").alias(f"{agg_col}_diff-1") for agg_col in agg_cols + bool_cols
]  # 1ステップ前の時間の値との差分
exprs += [
    pl.col(agg_col).diff(1).over("scene").alias(f"{agg_col}_diff1") for agg_col in agg_cols + bool_cols
]  # 1ステップ後の時間の値との差分
exprs += [pl.col(agg_col).mean().over("scene").alias(f"{agg_col}_mean") for agg_col in agg_cols]  # 同一シーンの平均値
exprs += [pl.col(agg_col).std().over("scene").alias(f"{agg_col}_std") for agg_col in agg_cols]  # 同一シーンの標準偏差
exprs += [pl.col(agg_col).max().over("scene").alias(f"{agg_col}_max") for agg_col in agg_cols]  # 同一シーンの最大値
exprs += [pl.col(agg_col).min().over("scene").alias(f"{agg_col}_min") for agg_col in agg_cols]  # 同一シーンの最小値

# train_test = train_test.sort(
#     # shiftと diffが時系列順に並んでいる必要があるためシーンごとに時間軸でソート
#     "scene",
#     "decisecond",
# ).with_columns(exprs)
train = train.sort("scene", "decisecond").with_columns(exprs)
test = test.sort("scene", "decisecond").with_columns(exprs)


In [ ]:
# 信号機情報
traffic_lights_dfs = []

# 信号機情報が入っているjsonファイルを1件ずつ読み込む
for f in tqdm(glob.glob("../../data/input/traffic_lights/*"), smoothing=True):
    if os.path.getsize(f) < 3:
        # ファイルサイズが3未満（からのファイル）の場合はスキップ
        # からのファイルはおそらく信号機が映っていない or 検出できなかったものと思われる
        continue
    data_id = f.split("/")[-1].split(".")[0]
    # jsonを読み込みフォルダ名からIDカラムを作成
    traffic_lights_dfs.append(pl.read_json(f).with_columns(pl.lit(data_id).alias("ID")))

# jsonから作成した個別のDataFrameを結合
traffic_lights_df = pl.concat(traffic_lights_dfs, how="vertical")
# bboxがListになっているので列に展開
bbox_df = traffic_lights_df["bbox"].list.to_struct(fields=[f"bbox_{i}" for i in range(4)]).struct.unnest()
traffic_lights_df = pl.concat([traffic_lights_df, bbox_df], how="horizontal").select(pl.all().exclude("bbox"))
traffic_lights_df


In [ ]:
# classの数を集計
traffic_lights_class_count_df = traffic_lights_df.pivot(
    values="class", index="ID", on="class", aggregate_function="len"
).fill_null(0)
traffic_lights_class_count_df


In [14]:
# 信号機情報をtrainとtestに追加
train = train.join(traffic_lights_class_count_df, how="left", on="ID")
test = test.join(traffic_lights_class_count_df, how="left", on="ID")


In [15]:
# 信号機の位置情報を活用した特徴量
train = train.with_columns(
    [
        # 信号機の検出数と車速の組み合わせ
        (pl.col("green") * pl.col("vEgo")).alias("green_light_speed"),
        (pl.col("red") * pl.col("vEgo")).alias("red_light_speed"),
        # 信号機の総検出数
        (pl.sum_horizontal(["green", "red", "yellow"])).alias("total_traffic_lights"),
    ]
)

test = test.with_columns(
    [
        # 信号機の検出数と車速の組み合わせ
        (pl.col("green") * pl.col("vEgo")).alias("green_light_speed"),
        (pl.col("red") * pl.col("vEgo")).alias("red_light_speed"),
        # 信号機の総検出数
        (pl.sum_horizontal(["green", "red", "yellow"])).alias("total_traffic_lights"),
    ]
)


In [16]:
# 元の順序に戻す
train = train.sort("index")
test = test.sort("index")


In [ ]:
print(len(train.columns), len(test.columns))


In [ ]:
train.head()


In [19]:
del_cols = ["ID", "scene", "decisecond", "index", "brake"] + cfg.target_cols
use_cols = list(set(train.columns) - set(del_cols))
use_cols.sort()

group_cols = ["scene"]


In [20]:
x_train = train.select(use_cols).to_pandas()
group = train.select(group_cols).to_pandas()
x_test = test.select(use_cols).to_pandas()


### 学習

In [21]:
# 学習関数
def lgb_cv_train(x_train, y_train, group):
    # 学習設定
    lgb_data = lgb.Dataset(
        x_train,
        label=y_train,
        # categorical_feature=cfg.categorical_features,
    )
    callbacks = [
        lgb.early_stopping(stopping_rounds=cfg.lgb.early_stopping_rounds),
        lgb.log_evaluation(cfg.lgb.log_evaluation_period),
    ]

    # 学習
    cv_results = lgb.cv(
        dict(cfg.lgb.params),
        lgb_data,
        folds=gkf.split(x_train, y_train, group),
        num_boost_round=cfg.lgb.num_boost_round,
        callbacks=callbacks,
        return_cvbooster=True,
        seed=cfg.seed,
    )

    return cv_results


# log 関数
def log_cv_results(cv_results, target_col):
    cvbooster = cv_results["cvbooster"]
    best_iteration = cvbooster.best_iteration
    best_score = cv_results["valid l1-mean"][best_iteration - 1]
    best_score_stdv = cv_results["valid l1-stdv"][best_iteration - 1]

    with open(f"{cfg.data.results_path}/{target_col}/log.txt", "w") as log_file:
        log_file.write("====== CV Score ======\n")
        log_file.write(f"best_iteration: {best_iteration}\n")
        log_file.write(f"best_score: {best_score}\n")
        log_file.write(f"best_score_stdv: {best_score_stdv}\n")

        log_file.write("\n====== params ======\n")
        log_file.write(OmegaConf.to_yaml(cfg, resolve=True))

        log_file.write("\n====== feature name ======\n")
        log_file.write(pprint.pformat(cvbooster.feature_name()[0]))


In [ ]:
# 実験結果格納用のディレクトリを作成
japan_tz = pytz.timezone("Asia/Tokyo")
cfg.run_time = datetime.now(japan_tz).strftime("%Y%m%d_%H%M%S")
create_dir(cfg.data.results_path)

# cv, predsリスト
cv_scores = []
target_preds = []

for target_col in cfg.target_cols:
    print(f"\n\n===== target_col: {target_col} =====\n\n")
    y_train = train.select(target_col).to_pandas()
    # 学習
    cv_results = lgb_cv_train(x_train, y_train, group)

    # 学習モデル保存
    cvbooster = cv_results["cvbooster"]
    create_dir(f"{cfg.data.results_path}/{target_col}")
    with open(f"{cfg.data.results_path}/{target_col}/model.pkl", "wb") as f:
        pickle.dump(cvbooster, f)

    # log
    log_cv_results(cv_results, target_col)

    # CV用のスコアを保存
    best_iteration = cv_results["cvbooster"].best_iteration
    best_score = cv_results["valid l1-mean"][best_iteration - 1]
    cv_scores.append(best_score)

    # テストコードの推論
    x_test = test.select(use_cols).to_pandas()
    y_preds = cvbooster.predict(x_test, num_iteration=cvbooster.best_iteration)
    y_pred = np.mean(y_preds, axis=0)
    target_preds.append(y_pred)

# # モデルのロード
# with open(f"{cfg.data.results_path}/model.pkl", "rb") as f:
#     loaded_cvbooster = pickle.load(f)

# cv_score
cv_score = np.mean(cv_scores)
print(f"cv_score: {cv_score}")

# submission
exprs = [pl.Series(target_preds[i]).alias(cfg.target_cols[i]) for i in range(len(cfg.target_cols))]
submission = sample_submission.with_columns(exprs)
submission.write_csv(f"{cfg.data.results_path}/submission_{cv_score:.4f}.csv")
submission.head()
